In [1]:
import numpy as np
import scipy as sp
import suite2p as s2p
import TwoPUtils as tpu
import reward_relative as rrel
import os

%matplotlib inline

%load_ext autoreload
%autoreload 2

ModuleNotFoundError: No module named 'spatial'

In [ ]:
from reward_relative.path_dict_firebird import path_dictionary as path_dict
# options: path_dict_josquin, path_dict_msosamac, etc.
path_dict

In [3]:
mouse = "GCAMP12"
basedir = os.path.join(path_dict['preprocessed_root'],mouse)
sbxdir = os.path.join(path_dict['sbx_root'],mouse) 

custom_nplanes = 1 # if needed to override empty optotune params

basedir, sbxdir

('/data/InVivoDA/GCAMP12', '/mnt/oak/InVivoDA/2P_Data/GCAMP12')

In [ ]:
file_list = rrel.sessions_dict.single_plane[mouse]

In [ ]:
for fn,f in enumerate(file_list):

    fullpath = os.path.join(basedir,f['date'],f['scene'],"%s_%03d_%03d" % (f['scene'], f['session'], f['scan']))
    scanpath = os.path.join(sbxdir,f['date'],f['scene'],"%s_%03d_%03d" % (f['scene'], f['session'], f['scan']))
    h5path = os.path.join(basedir,f['date'],f['scene'],"%s_%03d_%03d.h5" % (f['scene'], f['session'], f['scan']))

    scanmat, sbxfile = scanpath+'.mat', scanpath+'.sbx'

    info = tpu.scanner_tools.sbx_utils.loadmat(scanmat)
    if len(info['otwave'])>0:
        nplanes = info['otwave'].shape[0]
    else:
        nplanes = custom_nplanes
        print('"otwave" was empty; hardcoding %d planes' % nplanes)

    print('nplanes=',nplanes)

    h5name = tpu.scanner_tools.sbx_utils.sbx2h5(scanpath,output_name=h5path)

    ops = tpu.s2p.set_ops(d={'data_path': [os.path.split(fullpath)[0]],
                               'save_path0': fullpath,
                               'fast_disk':[],
                               'move_bin':True,
                               'two_step_registration':True,
                               'maxregshiftNR':10,
                               'nchannels':1,
                               'tau': 0.7,
                               'functional_chan':1,
                               'nimg_init': 2000,
                               'fs':info['frame_rate'],
                               'roidetect':True,
                               'input_format':"h5",
                               'h5py_key':'data',
                               'sparse_mode':True,
                               'threshold_scaling':1,
                                'nplanes':nplanes})

    ops=s2p.run_s2p(ops=ops)

    !rm {h5name} 